### [\_\_call\_\_](https://docs.python.org/2/reference/datamodel.html#emulating-callable-objects)

	
The functionality of \_\_call\_\_ is just like the overloaded operator of () in C++. 

- [What is a “callable” in Python?](http://stackoverflow.com/questions/111234/what-is-a-callable-in-python/115349#115349)
- [Python internals: how callables work?](http://eli.thegreenplace.net/2012/03/23/python-internals-how-callables-work/)

In [42]:
"""
A callable is anything that can be called.

object.__call__(self[, args...])

    Called when the instance is “called” as a function.
    If this method is defined, x(arg1, arg2, ...) == x.__call__(arg1, arg2, ...), for an instance x.

"""

class Foo:
    def __call__(self):
        print 'called'

foo_instance = Foo()
foo_instance() #this is calling the __call__ method
Foo()()        #this is calling the __call__ method

"""
That's also why you don't have foo = new int in Python: 

    you just make the class object return an instance of it on __call__. 
    The way Python solves this is very elegant in my opinion.
    
"""
print int('10')  , int.__call__('10')
print unicode(10), unicode.__call__(10)
print type(int), type(11) # <type 'type'> <type 'int'>

print 'Callable?'
print 'object:', object.mro(), callable(object)
print 'int:', int.mro(), callable(int), '      1:   ',  callable(1)
print 'str:', callable(str), "      'aa':", callable('aa')
print 'list:', callable(list), '    []:   ',  callable([])
print 'dict:', callable(dict), "    {}:", callable({})
print 'tuple:', callable(tuple), "    ():", callable(())

class Foo:
    pass

class Bar(object):
    pass

print type(Foo), type(Foo).__call__(Foo)
print type(Bar), type(Bar).__call__(Bar)

def foo(bar):
    return bar

print type(foo), type(foo).__call__(foo, 42)


called
called
10 10
10 10
<type 'type'> <type 'int'>
Callable?
object: [<type 'object'>] True
int: [<type 'int'>, <type 'object'>] True       1:    False
str: True       'aa': False
list: True     []:    False
dict: True     {}: False
tuple: True     (): False
<type 'classobj'> <__main__.Foo instance at 0x0000000003924748>
<type 'type'> <__main__.Bar object at 0x00000000039378D0>
<type 'function'> 42


##### Internals:

The built-in callable (**PyCallable_Check** in [**objects.c**](http://svn.python.org/projects/python/trunk/Objects/object.c) ) checks if the argument is either:
    
    1. an instance of a class with a __call__ method or
    2. is of a type that has a non null tp_call (c struct) member which indicates callability 
       (such as in functions, methods etc.)
       
##### In Python 3.x

This function callable() was first removed in Python 3.0 and then brought back in Python 3.2.


In [3]:
"""
Another example: a callable class as a decorator
"""

class Cached:
    def __init__(self, function):
        self.function = function
        self.cache = {}

    def __call__(self, *args):
        try: 
            return self.cache[args]
        except KeyError:
            ret = self.cache[args] = self.function(*args)
            return ret  
        
@Cached
def ack(x, y):
    return ack(x-1, ack(x, y-1)) if x*y else (x + y + 1) 

print ack(0,1)
print ack(1,0)
print ack(3,3)

2
2
53


In [6]:
"""
Example from standard library, file site.py, definition of built-in exit() and quit() functions:
"""

class Quitter(object):
    def __init__(self, name):
        self.name = name
    def __repr__(self):
        return 'Use %s() or %s to exit' % (self.name, eof)
    def __call__(self, code=None):
        # Shells like IDLE catch the SystemExit, but listen when their
        # stdin wrapper is closed.
        try:
            sys.stdin.close()
        except:
            pass
        raise SystemExit(code)
        
__builtin__.quit = Quitter('quit')
__builtin__.exit = Quitter('exit')


In [39]:
x=234

print x, 
_callable = False
for i in dir(x):
    if i.startswith("__call__"):
        print i
        _callable = True
        break
if not _callable:
    print 'Not callable'
        
print 'int', int.mro(), 
_callable = False
for i in dir(int):
    if i.startswith("__call__"):
        print i
        _callable = True
        break
if not _callable:
    print 'Not callable'
    
def add(x,y):
    return x+y

print 'add()',
_callable = False
for i in dir(add):
    if i.startswith("__call__"):
        print i
        _callable = True
        break
if not _callable:
    print 'Not callable'
    
print type(int), type(x), x.__class__

234 Not callable
int [<type 'int'>, <type 'object'>] Not callable
add() __call__
<type 'type'> <type 'int'> <type 'int'>


##### Application

A Callable is an object that has the `__call__` method. This means you can fake callable functions or do neat things like Partial Function Application where you take a function and add something that enhances it or fills in some of the parameters, returning something that can be called in turn (known as [**Currying**](https://en.wikipedia.org/wiki/Currying) in functional programming circles).